In [ ]:
import math

import matplotlib.pyplot as plt
import mpld3
import numpy as np
from IPython.display import HTML, display,SVG
import pydot
from pydrake.all import (
    AddMultibodyPlantSceneGraph,
    ControllabilityMatrix,
    DiagramBuilder,
    Linearize,
    LinearQuadraticRegulator,
    MeshcatVisualizer,
    AddDefaultVisualization,
    MultibodyPlant,
    Parser,
    Propeller,
    PropellerInfo,
    RigidTransform,
    Saturation,
    SceneGraph,
    Simulator,
    StartMeshcat,
    WrapToSystem,
    namedview,
)
from pydrake.examples import (
    AcrobotGeometry,
    AcrobotInput,
    AcrobotPlant,
    AcrobotState,
    QuadrotorGeometry,
    QuadrotorPlant,
    StabilizingLQRController,
)
from pydrake.solvers import MathematicalProgram, Solve

from underactuated import ConfigureParser, running_as_notebook
from underactuated.meshcat_utils import MeshcatSliders
from underactuated.quadrotor2d import Quadrotor2D, Quadrotor2DVisualizer
from underactuated.scenarios import AddFloatingRpyJoint

if running_as_notebook:
    mpld3.enable_notebook()

In [ ]:
# Start the visualizer (run this cell only once, each instance consumes a port)
meshcat = StartMeshcat()

In [ ]:
time_step = 1.0/60.0
test_mode = False

In [ ]:
def create_quadrotor(quadrotor_builder, quadrotor_plant, scene_graph):

    #### Define quadrotor MultiBodyPlant using Quadrotor .urdf and Propeller class
    (quad_model, ) = Parser(quadrotor_plant).AddModelsFromUrl(
        "package://drake/examples/quadrotor/quadrotor.urdf"
    )
    quad_body = quadrotor_plant.GetBodyByName("base_link")
    quad_frame = quadrotor_plant.GetFrameByName("base_link")

    ###### Change coordinates from quaternion to rpy ######
    AddFloatingRpyJoint(
        quadrotor_plant, quad_frame, quad_model,
        use_ball_rpy=False,
    )

    #finalize the plant
    quadrotor_plant.Finalize()

    # Define propellers
    prop_infos = []
    p_thrust_ratio = 1.0
    for i in range(1, 5):
        p_moment_ratio = -0.275 if i % 2 == 0 else 0.275
        p_pose = RigidTransform(p=[0.1750, 0, 0]) if i == 1 else \
                 RigidTransform(p=[0, 0.1750, 0]) if i == 2 else \
                 RigidTransform(p=[-0.1750, 0, 0]) if i == 3 else \
                 RigidTransform(p=[0, -0.1750, 0])
        prop_infos.append(PropellerInfo(quad_body.index(), p_pose, p_thrust_ratio, p_moment_ratio))
    props = quadrotor_builder.AddSystem(Propeller(prop_infos))
    quadrotor_builder.Connect(quadrotor_plant.get_body_poses_output_port(), 
                    props.get_body_poses_input_port())       
    quadrotor_builder.ExportInput(props.get_command_input_port(), "prop_command")
    quadrotor_builder.ExportOutput(quadrotor_plant.get_state_output_port(), "state")

    # Set up visualization in MeshCat
    meshcat.Delete()
    meshcat.ResetRenderMode()
    meshcat.SetProperty('/Background','visible',False)
    visualizer = MeshcatVisualizer.AddToBuilder(quadrotor_builder, scene_graph, meshcat)

    quadrotor_diagram = quadrotor_builder.Build()

    return quadrotor_diagram, visualizer


In [ ]:
# Setup builder and scene.
quadrotor_builder = DiagramBuilder()
quadrotor_plant, scene_graph = AddMultibodyPlantSceneGraph(quadrotor_builder, time_step = 1/60.0)
quadrotor_diagram, visualizer = create_quadrotor(quadrotor_builder, quadrotor_plant, scene_graph)
display(SVG(pydot.graph_from_dot_data(
    quadrotor_diagram.GetGraphvizString(max_depth=2))[0].create_svg()))

In [ ]:
# Setup builder and scene.
world_builder = DiagramBuilder()
quadrotor_builder = DiagramBuilder()
# define world multibody plant containing quadrotor body and obstacles
world_plant, scene_graph = AddMultibodyPlantSceneGraph(quadrotor_builder, time_step = 0.0)

# setup quadrotor plant
body = quadrotor_builder.AddSystem(MultibodyPlant(0.0))
parser = Parser(body)
# load models
(model_instance,) = parser.AddModelsFromUrl(
    "package://drake/examples/quadrotor/quadrotor.urdf"
)
AddFloatingRpyJoint(
    body,
    body.GetFrameByName("base_link"),
    model_instance,
    use_ball_rpy=False,
)
body.Finalize()

# connect propellers to the body
body_index = body.GetBodyByName("base_link").index()
L = 0.15  # Length of the arms (m).
kF = 1.0  # Force input constant.
kM = 0.0245  # Moment input constant.
prop_info = [
    PropellerInfo(body_index, RigidTransform([L, 0, 0]), kF, kM),
    PropellerInfo(body_index, RigidTransform([0, L, 0]), kF, -kM),
    PropellerInfo(body_index, RigidTransform([-L, 0, 0]), kF, kM),
    PropellerInfo(body_index, RigidTransform([0, -L, 0]), kF, -kM),
]
propellers = quadrotor_builder.AddSystem(Propeller(prop_info))
quadrotor_builder.Connect(
    propellers.get_output_port(),
    body.get_applied_spatial_force_input_port(),
)
quadrotor_builder.Connect(
    body.get_body_poses_output_port(),
    propellers.get_body_poses_input_port(),
)
quadrotor_builder.ExportInput(propellers.get_command_input_port(), "u")

# Clean up the Meshcat instance.
meshcat.Delete()
meshcat.DeleteAddedControls()
visualizer = MeshcatVisualizer.AddToBuilder(quadrotor_builder, scene_graph, meshcat)

quadrotor = quadrotor_builder.Build()

In [ ]:
# initial conditions
state_view = namedview("state", body.GetStateNames(False))
context = quadrotor.CreateDefaultContext()
nominal_state = state_view.Zero()
nominal_state.z_x = 1.0
context.SetContinuousState(nominal_state[:])

# get parameters
mass = body.CalcTotalMass(body.GetMyContextFromRoot(context))
gravity = body.gravity_field().gravity_vector()[2]
nominal_input = [-mass * gravity / 4] * 4

# setup controller
quadrotor.get_input_port().FixValue(context, nominal_input)
Q = np.diag(np.concatenate(([10] * 6, [1] * 6)))
R = np.eye(4)
controller = LinearQuadraticRegulator(quadrotor, context, Q, R)

# setup diagram
world_builder.AddSystem(quadrotor)
world_builder.AddSystem(controller)
world_builder.Connect(controller.get_output_port(), quadrotor.get_input_port())

# setup visualizer
# Build the diagram
quadrotor_control = world_builder.Build()
# Visualize the diagram.
display(SVG(pydot.graph_from_dot_data(
    quadrotor_control.GetGraphvizString(max_depth=2))[0].create_svg()))

# run simulation
simulator = Simulator(quadrotor_control)
simulator.Initialize()
simulator.set_target_realtime_rate(1.)
meshcat.StartRecording()
finish_time = 0.1 if test_mode else 2.0
simulator.AdvanceTo(finish_time)
meshcat.PublishRecording()